In [62]:
from bs4 import BeautifulSoup as beauty
import pandas as pd
import numpy as np
import os

In [12]:
list_2019_Vker_Hu =[i for i in os.listdir() if '2019' in i if 'ingatlanokHU_Vker' in i]
list_2019_Vker_Com = [i for i in os.listdir() if '2019' in i if 'ingatlanCOM_Vker' in i]
list_2019_Vker_Cen = [i for i in os.listdir() if '2019' in i if 'otthoncentrum_Vker' in i]

In [13]:
list_2019_Vker_Hu

['ingatlanokHU_Vker_2492019.csv',
 'ingatlanokHU_Vker_1892019.csv',
 'ingatlanokHU_Vker_3092019.csv',
 'ingatlanokHU_Vker_10102019.csv',
 'ingatlanokHU_Vker_14102019.csv',
 'ingatlanokHU_Vker_21102019.csv']

In [123]:
HU = pd.DataFrame(columns =['id','sqm','room','type','price','description','agency','date'])

In [124]:
for i in list_2019_Vker_Hu:
    html = open(i, 'r').read()
    l = beauty(html, 'lxml')
    for ii in range(len(l.find_all('div', attrs={'class': 'result-row'}))):
        value = []
        # ID
        value.append(l.find_all('div', attrs={'class': 'result-row'})[ii].find_all('a')[1]['href'].split('/')[-1])
        #sqm
        sqm = l.find_all('div', attrs={'class': 'result-row'})[ii].find_all(attrs={'class':'column-first'})
        if len(sqm) > 0:
            value.append(sqm[0].text.strip().replace('\n', ' ').split(' ')[1])
        else: value.append(np.NAN)
        #rooms
        room = l.find_all('div', attrs={'class': 'result-row'})[ii].find_all(attrs={'class':'column'})
        if len(room) > 0:
            value.append(room[0].text.strip().replace('\n', ' ').split(' ')[1])
        else: value.append(np.NAN)
        # type
        types = l.find_all('div', attrs={'class': 'result-row'})[ii].find_all(attrs={'class':'column'})
        if len(types) > 1:
            value.append(types[1].text.strip().replace('\n', ' ').split(' ')[2])
        else: value.append(np.NAN)
        # price
        p = l.find_all('div', attrs={'class': 'result-row'})[ii].find(attrs={'class':'result-price left'})
        if len(p) > 0:
            value.append(p.text.strip().replace('.', '').split(' ')[0])
        else: value.append(np.NAN)
        #description
        
        value.append(l.find_all('div', attrs={'class': 'result-row'})[ii].find(attrs={'class': 'result-short-description'}).text.strip())
        #agency
        age = l.find_all('div', attrs={'class': 'result-row'})[ii].find_all(attrs={'class':'office-logo-container right'})
        if len(age) > 0:
            value.append(age[0].find('a')['title'])
        else: value.append(np.NAN)
        #date
        value.append(i)
        HU.loc[len(HU), :] =  value

TypeError: object of type 'NoneType' has no len()

In [111]:
l.find_all('div', attrs={'class': 'result-row'})[3]#.find_all(attrs={'class':'office-logo-container right'})#[0].find('a')['title']

<div class="result-row">
<div class="result-item cf">
<div class="result-image left">
<img alt="Eladó Lakás Budapest V. Ker" height="120" src="https://simg.ingatlanok.hu/ZFJZKSZn/s-b3:-QwhyYlxfUpiYv1kPAxk9gKwLhX5DCHJ69/w:150_h:120_q:70/i/0ZCyy2Tgb_ekSc8BU6IIyJNDSNVSjjsv" width="150"/>
<a class="result-image-hover" href="javascript:;" onclick="openGallery(9603730, 0, 1);" title="Galéria megnyitása"></a>
</div>
<div class="result-content left">
<div class="result-title-price">
<div class="result-title left">
<a href="https://ingatlanok.hu/elado/lakas/budapest_v_ker/44-millio-ft;33-negyzetmeter;1-szoba;tegla-epitesu/9603730" target="_blank" title="Eladó Lakás Budapest V. Ker">Eladó Lakás Budapest V. Ker</a>
</div>
</div>
<div class="result-short-description">
<a href="https://ingatlanok.hu/elado/lakas/budapest_v_ker/44-millio-ft;33-negyzetmeter;1-szoba;tegla-epitesu/9603730" target="_blank">EGYETEMISTÁK - BEFEKTETŐK- ELSŐ LAKÁSUKAT KERESŐK FIGYELEM!!!
Budapest 3 nagy egyetemének ( BME, Co

In [125]:
HU

,id,sqm,room,type,price,description,agency,date
0,9737619,32,1,Tégla,39900000,"3 az 1-ben! Tökéletes befektetés, ahol a moder...",CDC Ingatlan IV.Ker. Szita Ingatlan Kft.,ingatlanokHU_Vker_2492019.csv
1,9737341,30,1,Tégla,37500000,"3 az 1-ben! Tökéletes befektetés, ahol a moder...",CDC Ingatlan IV.Ker. Szita Ingatlan Kft.,ingatlanokHU_Vker_2492019.csv
2,9729036,30,1,Tégla,37500000,"3 az 1-ben! Tökéletes befektetés, ahol a moder...",CDC Ingatlan IV.Ker. Szita Ingatlan Kft.,ingatlanokHU_Vker_2492019.csv
3,9603730,33,1,Tégla,44300000,EGYETEMISTÁK - BEFEKTETŐK- ELSŐ LAKÁSUKAT KERE...,NaN,ingatlanokHU_Vker_2492019.csv
4,9729050,32,1,Tégla,39900000,"3 az 1-ben! Tökéletes befektetés, ahol a moder...",Cdci Teréz krt-i iroda,ingatlanokHU_Vker_2492019.csv
5,9352021,26,1,Tégla,28600000,Budapest V. kerületének központi közkedvelt ré...,GREAT HOME,ingatlanokHU_Vker_2492019.csv
6,9286302,30,0,Tégla,43000000,Eladó az V. kerületben egyedülálló panorámával...,Gold House Ingatlaniroda,ingatlanokHU_Vker_2492019.csv
7,9726406,30,1,Tégla,37500000,"3 az 1-ben! Tökéletes befektetés, ahol a moder...",CDC Báthory,ingatlanokHU_Vker_2492019.csv
8,9726678,32,1,Tégla,39900000,"3 az 1-ben! Tökéletes befektetés, ahol a moder...",CDC Báthory,ingatlanokHU_Vker_2492019.csv
9,9726534,32,1,Tégla,39900000,"3 az 1-ben! Tökéletes befektetés, ahol a moder...",CDC Bokréta,ingatlanokHU_Vker_2492019.csv
